## Init Dependencies

In [1]:
import requests
import os
import pandas as pd
from datetime import datetime
import json
import numpy as np

### Workspace Setup

In [2]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

## Set Static Variables

In [3]:
URL = 'http://rsr.akvo.org/rest/v1/'
PROJECT_ID = '7950'
#PROJECT_ID = '7283'
#PROJECT_TYPE = 'grand_parent'
PROJECT_TYPE = 'parent'
#PROJECT_TYPE = 'child'
RSR_TOKEN = os.environ['RSR_TOKEN']
FMT = '/?format=json&limit=1'
FMT100 = '/?format=json&limit=100'

## Set Filter

In [4]:
#FILTER_DATE = '2017-01-01 - 2017-12-31'
FILTER_DATE = '2017'
FILTER_DATE_TYPE = 'yearly'
#FILTER_DATE_TYPE = 'semester'
FILTER_COUNTRY = ['Malawi','Mozambique','Zambia']
HTML_TYPE = 'table' #print or table

## Set Authentication

In [5]:
headers = {
    'content-type': 'application/json',
    'Authorization': RSR_TOKEN
}

## Helper Functions

In [6]:
def get_response(endpoint, param, value):
    uri = '{}{}{}&{}={}'.format(URL, endpoint, FMT100, param, value)
    print(get_time() + ' Fetching - ' + uri)
    data = requests.get(uri, headers=headers)
    data = data.json()
    return data

In [7]:
def get_time():
    now = datetime.now().time().strftime("%H:%M:%S")
    return now

In [8]:
def get_sibling_id(x):
    for k,v in x.items():
        return k

In [9]:
def get_report_type(ps,pe):
    rt = {'is_yearly':False}
    try:
        psm = ps.split('-')[1]
        pem = pe.split('-')[1]
        if psm == '01' and pem == '12':
            rt = {'is_yearly':True}
        if psm == '01' and pem == '01':
            rt = {'is_yearly':True}
    except:
        pass
    return rt

In [10]:
def get_dimension_country(dv):
    dp = dv['value'].split(' - ')
    dv = {}
    if dp[0].lower() in ['zambia','malawi','mozambique']:
        dv.update({
            'commodity':'',
            'country':dp[0],
            'has_commodity':False,
            'has_country':True
        })
    else:
        dv.update({
            'commodity':dp[0],
            'country':'',
            'has_commodity':False,
            'has_country':True
        })
    if len(dp) == 2:
        dv.update({
            'commodity':dp[0],
            'country':dp[1],
            'has_commodity':True,
            'has_country':True
        })
    return dv

## Find Related Project

In [11]:
related_project = get_response('related_project','related_project',PROJECT_ID)

10:55:31 Fetching - http://rsr.akvo.org/rest/v1/related_project/?format=json&limit=100&related_project=7950


In [12]:
related_project

{'count': 3,
 'next': None,
 'previous': None,
 'results': [{'id': 3778,
   'related_project': 7950,
   'related_project_name': 'Agricultural Productivity Programme for Southern Africa (APPSA)',
   'related_project_show_link': '<a href="/en/project/7950/">Agricultural Productivity Programme for Southern Africa (APPSA)</a>',
   'relation_label': '1 - Parent',
   'related_iati_id': '',
   'relation': '1',
   'project': 8019},
  {'id': 3777,
   'related_project': 7950,
   'related_project_name': 'Agricultural Productivity Programme for Southern Africa (APPSA)',
   'related_project_show_link': '<a href="/en/project/7950/">Agricultural Productivity Programme for Southern Africa (APPSA)</a>',
   'relation_label': '1 - Parent',
   'related_iati_id': '',
   'relation': '1',
   'project': 7924},
  {'id': 3523,
   'related_project': 7950,
   'related_project_name': 'Agricultural Productivity Programme for Southern Africa (APPSA)',
   'related_project_show_link': '<a href="/en/project/7950/">Agri

In [13]:
results_framework_list = [PROJECT_ID]
if PROJECT_TYPE == 'parent':
    results_framework_list = list(pd.DataFrame(related_project['results'])['project'])
if PROJECT_TYPE == 'grand_parent':
    parent_list = list(pd.DataFrame(related_project['results'])['project'])
    results_framwork_list = []
    for second_level in parent_list:
        second_list = get_response('related_project','related_project',second_level)
        try:
            second_list = list(pd.DataFrame(second_list['results'])['project'])
            for third_level in second_list:
                results_framework_list.append(third_level)
        except:
            pass

In [14]:
results_framework_list

[8019, 7924, 7858]

### Trace All Children (Alternative)

In [15]:
def trace_all_childrens(project_id, all_childs, level):
    related = get_response('related_project','related_project',project_id)        
    if len(related['results']) > 0:
        for result in related['results']:
            level = level + 1
            print(level)
            all_childs.append(result)
            trace_all_childrens(result['project'], all_childs, level)
    return all_childs

### Trace All Children Results(Alternative)

In [16]:
def trace_all_results(all_childs):
    results_framework = []
    for child in all_childs:
        res = get_response('results_framework','project',child['project'])
        for rf in res['results']:
            results_framework.append(rf)
    return results_framework

### Concat Results Frameworks (Alternative)

In [17]:
def trace_onechildren():
    results_framework = []
    for i, rf in enumerate(results_framework_list):
        result_framework = get_response('results_framework','project',rf)['results']
        if i == 0:
            results_framework = result_framework
        else:
            for res in result_framework:
                results_framework.append(res)
    return results_framework

### Only Parents

In [18]:
results_framework = []
def no_trace():
    results_framework = get_response('results_framework','project',PROJECT_ID)['results']
    return results_framework

### Choose Trace Level

In [19]:
if PROJECT_TYPE == 'child':
    results_framework = no_trace()
if PROJECT_TYPE == 'parent':
    results_framework = trace_onechildren()
if PROJECT_TYPE == 'grand_parent':
    #all_childrens = trace_all_childrens(PROJECT_ID, [],1)
    results_framework = trace_onechildren()

10:55:32 Fetching - http://rsr.akvo.org/rest/v1/results_framework/?format=json&limit=100&project=8019
10:55:38 Fetching - http://rsr.akvo.org/rest/v1/results_framework/?format=json&limit=100&project=7924
10:55:43 Fetching - http://rsr.akvo.org/rest/v1/results_framework/?format=json&limit=100&project=7858


## Begin Transformations

In [20]:
results_framework = pd.DataFrame(results_framework)

### Generate List of All Objects

In [21]:
results_framework['child_projects'] = results_framework['child_projects'].apply(get_sibling_id)

In [22]:
if PROJECT_TYPE == 'grand_parent':
    results_framwork = results_framework[results_framework['child_projects'].isnull()].reset_index().drop(columns=['index'])

In [23]:
results_framework = results_framework.to_dict('records')

In [24]:
indicators = []
periods = []
dimension_names = []
dimension_values = []
disaggregations = []
disaggregation_targets = []
for result_framework in results_framework:
    rf_id = {'result':result_framework['id']}
    rf_project = {'project':result_framework['project_title']}
    rf_title = {'project_title':result_framework['title']}
    for indicator in result_framework['indicators']:
        indicator_id = indicator['id']
        indicator_title = {'indicator':indicator['title']}
        for period in indicator['periods']:
            is_yearly = get_report_type(period['period_start'],period['period_end'])
            period.update(rf_title)
            period.update(rf_project)
            period.update(is_yearly)
            period.update(rf_id)
            period.update({'indicator':indicator_id})
            periods.append(period)
            for data in period['data']:
                if len(data) > 0:
                    for disaggregation in data['disaggregations']:
                        disaggregation.update({'period':period['id']})
                        disaggregation.update({'parent_period':period['parent_period']})
                        disaggregation.update({'result_id':result_framework['id']})
                        disaggregation.update(rf_title)
                        disaggregation.update(rf_project)
                        disaggregation.update({'indicator_id':indicator_id})
                        disaggregation.update(indicator_title)
                        disaggregations.append(disaggregation)
            if len(period['disaggregation_targets']) > 0:
                for disaggregation_target in period['disaggregation_targets']:
                    disaggregation_target.update({'period':period['id']})
                    disaggregation_target.update({'parent_period':period['parent_period']})
                    disaggregation_target.update({'indicator_id':indicator_id})
                    disaggregation_target.update(indicator_title)
                    disaggregation_target.update({'result_id':result_framework['id']})
                    disaggregation_target.update(rf_title)
                    disaggregation_target.update(rf_project)
                    disaggregation_targets.append(disaggregation_target)
        #del indicator['periods']
        for dimension_name in indicator['dimension_names']:
            for dimension_value in dimension_name['values']:
                dimension_value.update(rf_id)
                dimension_update = get_dimension_country(dimension_value)
                dimension_value.update(dimension_update)
                dimension_values.append(dimension_value)
            # del dimension_name['values']
            dimension_name.update(rf_id)
            dimension_name.update({'indicator':indicator_id})
            dimension_names.append(dimension_name)
        #del indicator['dimension_names']
        indicators.append(indicator)

### Merge Dimension Values & Disaggregations

In [25]:
dimension_values = pd.DataFrame(dimension_values).groupby(['id']).first().reset_index()

## DN

In [26]:
dimension_names = pd.DataFrame(dimension_names).groupby(['id']).first()
dimension_names = dimension_names.drop(columns=['values','project','parent_dimension_name','result','indicator']).reset_index()
dimension_names = dimension_names.rename(columns={'id':'dimension_id','name':'dimension_name'})
dimension_values = dimension_values.merge(dimension_names,left_on='name',right_on='dimension_id',how='outer')

In [27]:
def fill_country(x):
    country = x['country']
    if country.lower() not in  ['zambia','malawi','mozambique']:
        country = x['project'].split(' ')[1]
    return country

### Disaggregation Values

In [28]:
remove_columns = [
    'created_at',
    'last_modified_at',
    'numerator',
    'denominator',
    'narrative',
    'dimension_value',
    'incomplete_data',
    'update',
    'dimension_name_disaggregation',
    'dimension_id'
]
rename_columns = {
    'value_dimension_values': 'disaggregation_name',
    'value_disaggregation': 'disaggregation_value',
    'dimension_name_dimension_values':'dimension_name'
}
fill_values = {
    'disaggregation_value':0,
    'incomplete_data':True
}
column_order = ['parent_dimension_value',
                'parent_period',
                'result',
                'country',
                'dimension_'
                'dimension_name',
                'disaggregation_name',
                'id',
                'commodity',
                'has_country',
                'has_commodity',
                'incomplete_data',
                'disaggregation_value',
                'period',
                'project_title',
                'project',
                'indicator',
                'indicator_id'
]
disaggregation_value = ['disaggregation_value']
disaggregations_merged = pd.DataFrame(disaggregations).drop(columns=['id']).merge(
    dimension_values, 
    how='outer', 
    left_on='dimension_value', 
    right_on='id', 
    suffixes=('_disaggregation','_dimension_values'))
disaggregations_merged = disaggregations_merged.drop(columns=remove_columns)
disaggregations_merged = disaggregations_merged.rename(columns=rename_columns)
disaggregations_merged = disaggregations_merged.fillna(value=fill_values)
disaggregations_merged['value'] = disaggregations_merged['disaggregation_value'].apply(lambda x:int(float(x)))
disaggregations_merged = disaggregations_merged.drop(columns=['disaggregation_value'])
disaggregations_merged = disaggregations_merged.dropna(subset=['parent_period'])
disaggregations_merged['parent_period'] = disaggregations_merged['parent_period'].astype(int)
disaggregations_merged['period'] = disaggregations_merged['period'].apply(lambda x: int(float(x)))
disaggregations_merged['indicator_id'] = disaggregations_merged['indicator_id'].apply(lambda x: int(float(x)))
disaggregations_merged['type'] = 'Cumulative Actual Values'
disaggregations_merged['country'] = disaggregations_merged.apply(fill_country , axis = 1)

#### Country R&D

In [29]:
if PROJECT_ID == '7282':
    disaggregations_merged['project'] = 'APPSA Zambia'
    disaggregations_merged['commodity'] = disaggregations_merged['commodity'].apply(lambda x: x.replace('-',' ').title())
    disaggregations_merged['commodity'] = disaggregations_merged['commodity'].apply(lambda x: "Country Project" if x == "" else x)
    disaggregations_merged['country'] = disaggregations_merged.apply(fill_country, axis=1)
    disaggregations_merged = disaggregations_merged.drop(columns=['project'])

### Target Values

In [30]:
targets = pd.DataFrame(disaggregation_targets).fillna(0).drop(columns=['id']).merge(
    dimension_values, 
    how='outer', 
    left_on='dimension_value', 
    right_on='id', 
    suffixes=('_target','_dimension_values'))
targets = targets.dropna(subset=['parent_period','id'])
targets['has_commodity'] = False
targets['has_country'] = True
fill_values = {
    'value_target':0
}
targets  = targets.fillna(value=fill_values)
targets['value'] =  targets['value_target'].apply(lambda x:int(float(x)))
targets = targets.drop(columns=['value_target'])
integer_list = ['id','name','parent_dimension_value','result','parent_period','dimension_value','dimension_id','period']
targets[integer_list] = targets[integer_list].astype(int)
rename_columns = {'value_dimension_values': 'disaggregation_name'}
unavailable_columns = ['disaggregation_value', 'incomplete_data','dimension_value','dimension_dimension_name']
column_order = [x for x in column_order if x not in unavailable_columns]
column_order = column_order + ['value','dimension_name','dimension_id']

In [31]:
targets = targets.rename(columns=rename_columns)[column_order]
targets['indicator_id'] = targets['indicator_id'].apply(lambda x: int(float(x)))
targets['type'] = 'Y4 RCoLs Targets'
if PROJECT_ID == '7282':
    targets['project'] = 'APPSA Zambia'
    targets['commodity'] = targets['commodity'].apply(lambda x: x.replace('-',' ').title())
    targets['commodity'] = targets['commodity'].apply(lambda x: "Country Project" if x == "" else x)
    targets['country'] = targets.apply(fill_country, axis=1)
    targets = targets.drop(columns=['project'])
else:
    targets['country'] = ''
    targets['country'] = targets.apply(fill_country, axis = 1)

In [32]:
disaggregations_merged = disaggregations_merged.rename(columns={'name':'dimension_id'})

In [33]:
order_columns = ['id','result','indicator_id','dimension_id','project_title','indicator','dimension_name','commodity','period','country','type','value']

In [34]:
targets = targets[order_columns]
disaggregations_merged = disaggregations_merged[order_columns]

## Ajax

### Redefining Period

In [141]:
periods_short = pd.DataFrame(periods)
periods_short = periods_short[['id','is_yearly','period_start','period_end']]
periods_short['period_date'] = periods_short['period_start'] + ' - ' + periods_short['period_end']

### Defining Values

In [142]:
ajax = pd.concat([disaggregations_merged,targets],sort=False)
ajax = ajax.sort_values(by=['result','indicator_id','dimension_id','id'])
ajax = ajax.merge(periods_short,how='inner',left_on='period',right_on='id',suffixes=('_data','_period')).sort_values(['id_data','indicator_id','dimension_name'])
remove_columns = [
    'period_end',
    'period_start',
    'id_period',
    'period',
    'is_yearly'
]
ajax = ajax[pd.notnull(ajax['period_end'])]

In [143]:
ajax['cumulative'] = ajax['period_end'].apply(lambda x: True if x.split('-')[1] == '12' else False)
ajax = ajax[ajax['cumulative'] == True].drop(columns=['cumulative'])
ajax = ajax.drop(columns=remove_columns)
if FILTER_DATE_TYPE == 'yearly':
    ajax['year'] = ajax['period_date'].apply(lambda x: x.split(' - ')[0].split('-')[0])
    ajax = ajax[ajax['year'] == FILTER_DATE].drop(columns=['year','period_date'])
else:
    pass
    #ajax = ajax[ajax['period_date'] == FILTER_DATE].drop(columns=['period_date'])

In [144]:
order_columns = ['result','project_title','indicator_id','indicator','target_value','actual_value','dimension_id','dimension_name','commodity','type','country','value','id_data']
ajax = ajax[order_columns]
ajax_group = ['result','project_title','indicator_id','indicator','target_value','actual_value','dimension_id','dimension_name','id_data','commodity','country','type']
ajax_sort = ['result','indicator_id','dimension_id','id_data']
ajax = ajax.groupby(ajax_group).first()

In [145]:
ajax = ajax.unstack('type').unstack('country').sort_values(ajax_sort)

In [146]:
ajax.shape

(157, 6)

In [149]:
ajax

value  \
type                                                                                                                                                                                                        Cumulative Actual Values   
country                                                                                                                                                                                                                       Malawi   
project_title                                      indicator                                          target_value actual_value dimension_name           commodity                                                                     
Project Development Objective: Increase the ava... PDO 1 Number of technologies that are being mad... 27           115.00       Type of technology       Number of improved seed varieties                                         0   
                                                                                                                                                         Maize                                                                     0   
                                                                                                                                                         Rice                                                                      0   
                                                                                                                                                         Legumes                                                                   0   
                                                                                                                                                         Cassava                                                                   0   
                                                                                                                                                         Number of improved agronomic, pest and disease,...                        0   
                                                                                                                                                         Number of improved post-harvest storage, labor-...                        0   
                                                   PDO 2 Proportion of Lead Farmers in targeted ar... 80           0            Gender RCoL              Male                                                                      0   
                                                                                                                                                         Female                                                                    0   
                                                   PDO 3 Number of technologies generated or promo... 7            28.00        Type of technology       Number of improved seed varieties                                         0   
                                                                                                                                                         Maize                                                                     0   
                                                                                                                                                         Rice                                                                      0   
                                                                                                                                                         Legumes                                                                   0   
                                                                                                                                                         Cassava                                                                   0   
                                               

In [150]:
ajax = pd.DataFrame(ajax['value'].to_records())
ajax = ajax.rename(columns={
    "('Cumulative Actual Values', 'Malawi')": "CA-MW",
    "('Cumulative Actual Values', 'Mozambique')": "CA-MZ",
    "('Cumulative Actual Values', 'Zambia')": "CA-ZA",
    "('Y4 RCoLs Targets', 'Malawi')":"TG-MW",
    "('Y4 RCoLs Targets', 'Mozambique')":"TG-MZ",
    "('Y4 RCoLs Targets', 'Zambia')":"TG-ZA"
})
ajax['TG-TTL'] = ajax['TG-MW'] + ajax['TG-MZ'] + ajax['TG-ZA']
ajax['CA-TTL'] = ajax['CA-MW'] + ajax['CA-MZ'] + ajax['CA-ZA']

In [151]:
ajax

,project_title,indicator,target_value,actual_value,dimension_name,commodity,CA-MW,CA-MZ,CA-ZA,TG-MW,TG-MZ,TG-ZA,TG-TTL,CA-TTL
0,Project Development Objective: Increase the av...,PDO 1 Number of technologies that are being ma...,27,115.00,Type of technology,Number of improved seed varieties,0,51,0,0,23,0,23,51
1,Project Development Objective: Increase the av...,PDO 1 Number of technologies that are being ma...,27,115.00,Type of technology,Maize,0,12,0,0,5,0,5,12
2,Project Development Objective: Increase the av...,PDO 1 Number of technologies that are being ma...,27,115.00,Type of technology,Rice,0,10,0,0,6,0,6,10
3,Project Development Objective: Increase the av...,PDO 1 Number of technologies that are being ma...,27,115.00,Type of technology,Legumes,0,23,0,0,12,0,12,23
4,Project Development Objective: Increase the av...,PDO 1 Number of technologies that are being ma...,27,115.00,Type of technology,Cassava,0,6,0,0,0,0,0,6
5,Project Development Objective: Increase the av...,PDO 1 Number of technologies that are being ma...,27,115.00,Type of technology,"Number of improved agronomic, pest and disease...",0,19,0,0,2,0,2,19
6,Project Development Objective: Increase the av...,PDO 1 Number of technologies that are being ma...,27,115.00,Type of technology,"Number of improved post-harvest storage, labor...",0,45,0,0,2,0,2,45
7,Project Development Objective: Increase the av...,PDO 2 Proportion of Lead Farmers in targeted a...,80,0,Gender RCoL,Male,0,0,0,0,52,0,52,0
8,Project Development Objective: Increase the av...,PDO 2 Proportion of Lead Farmers in targeted a...,80,0,Gender RCoL,Female,0,0,0,0,28,0,28,0
9,Project Development Objective: Increase the av...,PDO 3 Number of technologies generated or prom...,7,28.00,Type of technology,Number of improved seed varieties,0,10,0,0,6,0,6,10


### Defining Attribute Values

In [167]:
array = pd.concat([disaggregations_merged,targets],sort=False)
array = array.sort_values(by=['result','indicator_id','dimension_id','id'])
array = array.merge(periods_short,how='inner',left_on='period',right_on='id',suffixes=('_data','_period')).sort_values(['id_data','indicator_id','dimension_name'])

In [169]:
array

,id_data,result,indicator_id,dimension_id,project_title,indicator,dimension_name,commodity,period,country,type,value,id_period,is_yearly,period_start,period_end,target_value,actual_value,period_date
0,1291,36998,80750,554,Project Development Objective: Increase the av...,PDO 1 Number of technologies that are being ma...,Type of technology,Number of improved seed varieties,287882,Mozambique,Cumulative Actual Values,51,287882,True,2017-01-01,2017-12-31,27,115.00,2017-01-01 - 2017-12-31
1,1291,36998,80750,554,Project Development Objective: Increase the av...,PDO 1 Number of technologies that are being ma...,Type of technology,Number of improved seed varieties,287882,Mozambique,Y4 RCoLs Targets,23,287882,True,2017-01-01,2017-12-31,27,115.00,2017-01-01 - 2017-12-31
14,1291,36998,80750,554,Project Development Objective: Increase the av...,PDO 1 Number of technologies that are being ma...,Type of technology,Number of improved seed varieties,287878,Mozambique,Y4 RCoLs Targets,0,287878,True,2013-01-01,2013-12-31,,,2013-01-01 - 2013-12-31
21,1291,36998,80750,554,Project Development Objective: Increase the av...,PDO 1 Number of technologies that are being ma...,Type of technology,Number of improved seed varieties,287879,Mozambique,Y4 RCoLs Targets,0,287879,True,2014-01-01,2014-12-31,,,2014-01-01 - 2014-12-31
34,1291,36998,80752,554,Project Development Objective: Increase the av...,PDO 3 Number of technologies generated or prom...,Type of technology,Number of improved seed varieties,287896,Mozambique,Cumulative Actual Values,10,287896,True,2017-01-01,2017-12-31,7,28.00,2017-01-01 - 2017-12-31
35,1291,36998,80752,554,Project Development Objective: Increase the av...,PDO 3 Number of technologies generated or prom...,Type of technology,Number of improved seed varieties,287896,Mozambique,Y4 RCoLs Targets,6,287896,True,2017-01-01,2017-12-31,7,28.00,2017-01-01 - 2017-12-31
48,1291,36998,80752,554,Project Development Objective: Increase the av...,PDO 3 Number of technologies generated or prom...,Type of technology,Number of improved seed varieties,287892,Mozambique,Y4 RCoLs Targets,0,287892,True,2013-01-01,2013-12-31,,,2013-01-01 - 2013-12-31
61,1291,36998,80757,554,Intermediate result 1: Improved collaborative ...,8. Total number of improved technologies forma...,Type of technology,Number of improved seed varieties,287938,Mozambique,Cumulative Actual Values,51,287938,True,2017-01-01,2017-12-31,7,115.00,2017-01-01 - 2017-12-31
62,1291,36998,80757,554,Intermediate result 1: Improved collaborative ...,8. Total number of improved technologies forma...,Type of technology,Number of improved seed varieties,287938,Mozambique,Y4 RCoLs Targets,5,287938,True,2017-01-01,2017-12-31,7,115.00,2017-01-01 - 2017-12-31
75,1291,36998,80757,554,Intermediate result 1: Improved collaborative ...,8. Total number of improved technologies forma...,Type of technology,Number of improved seed varieties,287934,Mozambique,Y4 RCoLs Targets,0,287934,True,2013-01-01,2013-12-31,,,2013-01-01 - 2013-12-31


In [153]:
array['variable'] = array.apply(lambda x: {
    'id':x['id_data'],
    'result':x['result'],
    'country':x['country'],
    'type':x['type'],
    'period':x['period'],
    'date':x['period_date'],
    'indicator_id':x['indicator_id'],
    'indicator_name':x['indicator'],
    'dimension':x['dimension_id'],
    'dimension_name':x['dimension_name'],
    'commodity':x['commodity'],
    'value': x['value'],
    'target_value':x['target_value'],
    'actual_value':x['actual_value']
},axis=1)
array = array[pd.notnull(array['period_end'])]

In [154]:
array['cumulative'] = array['period_end'].apply(lambda x: True if x.split('-')[1] == '12' else False)
array = array[array['cumulative'] == True].drop(columns=['cumulative'])
if FILTER_DATE_TYPE == 'yearly':
    array['year'] = array['period_date'].apply(lambda x: x.split(' - ')[0].split('-')[0])
    array = array[array['year'] == FILTER_DATE].drop(columns=['year','period_date'])
else:
    pass
    #ajax = ajax[ajax['period_date'] == FILTER_DATE].drop(columns=['period_date'])

In [155]:
array_group =  ['result','project_title','indicator_id','indicator','dimension_id','dimension_name','id_data','commodity','country','type']
array_sort = ['result','indicator_id','dimension_id','id_data']

In [156]:
def group_attribute(a):
    print(a)
    data = []
    for b in a:
        data.append(b)
    return(data)

In [157]:
array = array.groupby(array_group)['variable'].apply(group_attribute).reset_index()
array = array.groupby(array_group).first().unstack('type').unstack('country').sort_values(array_sort)

In [158]:
array

variable  \
type                                                                                                                                                                                                                                                 Cumulative Actual Values   
country                                                                                                                                                                                                                                                                Malawi   
result project_title                                      indicator_id indicator                                          dimension_id dimension_name           id_data commodity                                                                                               
36998  Project Development Objective: Increase the ava... 80750        PDO 1 Number of technologies that are being mad... 554          Type of technology       1291    Number of improved seed varieties                                                                 NaN   
                                                                                                                                                                1292    Maize                                                                                             NaN   
                                                                                                                                                                1293    Rice                                                                                              NaN   
                                                                                                                                                                1294    Legumes                                                                                           NaN   
                                                                                                                                                                1295    Cassava                                                                                           NaN   
                                                                                                                                                                1296    Number of improved agronomic, pest and disease,...                                                NaN   
                                                                                                                                                                1297    Number of improved post-harvest storage, labor-...                                                NaN   
                                                          80751        PDO 2 Proportion of Lead Farmers in targeted ar... 557          Gender RCoL              1308    Male                                                                                              NaN   
                                                                                                                                                                1309    Female                                                                                            NaN   
                                                          80752        PDO 3 Number of technologies generated or promo... 554          Type of technology       1291    Number of improved seed varieties                                                                 NaN   
                                                                                                                                                                1292    Maize                                                                                             NaN   
                                                                                                                                                                1293  

In [159]:
array = array.groupby(level=[1,3,5,7],sort=False).first()
array = pd.DataFrame(array['variable'].to_records())
array = array.rename(columns={
    "('Cumulative Actual Values', 'Malawi')": "CA-MW-D",
    "('Cumulative Actual Values', 'Mozambique')": "CA-MZ-D",
    "('Cumulative Actual Values', 'Zambia')": "CA-ZA-D",
    "('Y4 RCoLs Targets', 'Malawi')":"TG-MW-D",
    "('Y4 RCoLs Targets', 'Mozambique')":"TG-MZ-D",
    "('Y4 RCoLs Targets', 'Zambia')":"TG-ZA-D"
})

In [160]:
array = array.drop(columns=['project_title','indicator','commodity','dimension_name'])
array = pd.merge(ajax, array, left_index=True, right_index=True)

In [161]:
array = array.replace({np.nan: None})

In [162]:
project = list(pd.DataFrame(results_framework).groupby('title').first().reset_index()['title'])

In [163]:
indicator_title = list(pd.DataFrame(indicators).groupby('title').first().reset_index()['title'])

In [164]:
indicator_var = pd.DataFrame(indicators).groupby('title').first().reset_index()[['title','id','description']]

In [1]:
array['CA-MW-D'][1]

NameError: name 'array' is not defined